# Generador Mock Ventas

In [6]:
# Importaciones
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

In [7]:
# Configuración de los parámetros para el generador

# Parámetros
n_transactions = 300000  # número de transacciones (ventas)
start_date = datetime(2024, 1, 1)
end_date = datetime(2025, 1, 31)
delta_days = (end_date - start_date).days + 1

# Función para generar una fecha aleatoria en el rango
def random_date():
    random_days = np.random.randint(0, delta_days)
    return start_date + timedelta(days=int(random_days))

# Distribución de tiendas: 1,2,3 con 0.3 cada uno y 4 (outlet) con 0.1
tiendas = [1, 2, 3, 4]
tienda_probs = [0.3, 0.3, 0.3, 0.1]

# Función para determinar el número de artículos en la venta
def random_items_count():
    r = random.random()
    if r < 0.70:
        return 1
    elif r < 0.90:
        return 2
    elif r < 0.95:
        return 3
    elif r < 0.98:
        return 4
    elif r < 0.99:
        return 5
    else:
        return random.randint(6, 10)

# Distribución de talles: 1:10%, 2:20%, 3:40%, 4:20%, 5:10%
talles = [1, 2, 3, 4, 5]
talle_probs = [0.1, 0.2, 0.4, 0.2, 0.1]

In [8]:
# Generar transacciones / Facturas

# Crear una lista para guardar las transacciones (a nivel factura)
transactions = []
for i in range(n_transactions):
    tienda = np.random.choice(tiendas, p=tienda_probs)
    fecha = random_date()
    n_items = random_items_count()
    transactions.append({
        "tienda": tienda,
        "fecha": fecha,
        "n_items": n_items,
        "trans_id": i  # id de transacción
    })

# Crear DataFrame y ordenar por tienda y fecha
# Asignar número de factura secuencial por tienda.
df_trans = pd.DataFrame(transactions)
df_trans = df_trans.sort_values(by=["tienda", "fecha", "trans_id"]).reset_index(drop=True)

# Inicializar diccionario para guardar el número inicial de factura para cada tienda
invoice_start = {}
for tienda in tiendas:
    if tienda == 4:
        invoice_start[tienda] = random.randint(2000, 3000)
    else:
        invoice_start[tienda] = random.randint(5000, 10000)

# Asignar números de factura por tienda
current_invoice = {tienda: invoice_start[tienda] for tienda in tiendas}
facturas = []
for idx, row in df_trans.iterrows():
    tienda = row["tienda"]
    factura = current_invoice[tienda]
    facturas.append(factura)
    current_invoice[tienda] += 1

df_trans["factura"] = facturas

In [9]:
# Generar líneas de venta

# Expandir cada transacción en las líneas de venta (cada línea es un artículo vendido)
sales_rows = []
for idx, row in df_trans.iterrows():
    for line in range(1, int(row["n_items"]) + 1):
        sales_rows.append({
            "empresa": "",
            "codigoarticulo": random.randint(1, 500),
            "talle": np.random.choice(talles, p=talle_probs),
            "codigodeposito": row["tienda"],
            "tipocomprobante": "FB",
            "numerocomprobante": row["factura"],
            "fechacomprobante": row["fecha"].strftime("%Y-%m-%d"),
            "hora": "00:00:00",
            "linea": line,
            "articulos": "",
            "cantidad": random.randint(1, 5),
            "preciounitario": "",
            "preciototal": "",
            "descuento": "",
            "deposito": ""
        })

df_sales = pd.DataFrame(sales_rows)

In [10]:
# Guardar el mock data de ventas en un CSV
output_file = "MOCK_DATA_Ventas.csv"
df_sales.to_csv(output_file, index=False)

print(f"Archivo guardado en: {output_file}")

Archivo guardado en: MOCK_DATA_Ventas.csv
